In [15]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path

In [17]:
path = Path.cwd()
data_path = path /'data'
train_path = data_path /'MINDsmall_train'

In [19]:
behavior = pd.read_table(train_path/'behaviors.tsv',
                         header=None,
                         names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

news = pd.read_table(train_path/'news.tsv',
                     header=None,
                     names=['id', 'category', 'subcategory', 'title', 'abstract', 'url',
                            'title_entities', 'abstract_entities'])


entity_embedding = pd.read_table(train_path/'entity_embedding.vec', header=None)
entity_embedding['vector'] = entity_embedding.iloc[:, 1:101].values.tolist()
entity_embedding = entity_embedding[[0, 'vector']].rename(columns={0: "entity"})


relation_embedding = pd.read_table(train_path/'relation_embedding.vec', header=None)
relation_embedding['vector'] = relation_embedding.iloc[:,1:101].values.tolist()
relation_embedding = relation_embedding[[0, 'vector']].rename(columns={0: "relation"})

In [21]:
behavior['time'] = pd.to_datetime(behavior['time'], format = "%m/%d/%Y %I:%M:%S %p")

In [22]:
print('Time Min and Max : {} - {}'.format(behavior['time'].min(), behavior['time'].max()))

Time Min and Max : 2019-11-09 00:00:19 - 2019-11-14 23:59:13


In [23]:
news.head()

,id,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N48100,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N54564,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://www.msn.com/en-us/health/weightloss/50...,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N2942,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N14733,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
4,N31455,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
